In [ ]:
### import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from datetime import datetime
from sas7bdat import SAS7BDAT
###from Random_Forest_Package_v1 import RFM_Shawn as RFM_CLF
from lib.Gradient_Boosting_Package_wyatt import GBM_Shawn as GBM_CLF
import pytz
import sas7bdat
import os,shutil,xlsxwriter

#pd.set_option('display.height', 1000)
#pd.set_option('display.width', 1000)
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 500)
### data preparation

In [ ]:
### data preparation
print('Data load strs at '+ datetime.now(pytz.timezone('Asia/Hong_Kong')).strftime('%y-%m-%d %H:%M:%S %z'))

csv_train='/Users/wyatt/Documents/Docs-wMBP15/SAS/stk_model/model.Wyatt_stock_train.csv'
#csv_train='/Users/wyatt/Documents/Docs-wMBP15/SAS/stk_model/model.Wyatt_stock_valid.csv'
csv_valid='/Users/wyatt/Documents/Docs-wMBP15/SAS/stk_model/model.Wyatt_stock_valid.csv'

fn_idc=csv_train.split('/')[-1].split('.')[-2]
input_train=pd.DataFrame(pd.read_csv(csv_train)).fillna(0)[:3000]
input_valid=pd.DataFrame(pd.read_csv(csv_valid)).fillna(0)[:1000]
#input_train=input_train.fillna(0)
#input_valid=input_valid.fillna(0)
print('Data load ends at '+ datetime.now(pytz.timezone('Asia/Hong_Kong')).strftime('%y-%m-%d %H:%M:%S %z'))
print('Data as of '+ datetime.now(pytz.timezone('Asia/Hong_Kong')).strftime('%y-%m-%d %H:%M:%S %z'))
print("#------------------- Separate Line 1 -------------------# ")
print("Train Data : ")
#input_train.info()
print("#------------------- Separate Line 2 -------------------# ")
print("Valid Data : ")
#input_valid.info()
print("#------------------- Separate Line 3 -------------------# ")

In [ ]:
print('Model train strs at '+ datetime.now(pytz.timezone('Asia/Hong_Kong')).strftime('%y-%m-%d %H:%M:%S %z'))

gb_clf=GBM_CLF(mode='superfast', df_train=input_train, df_valid=input_valid, str_resp='response',str_id='cust_id')
gb_clf._training()

print('Model train ends at '+ datetime.now(pytz.timezone('Asia/Hong_Kong')).strftime('%y-%m-%d %H:%M:%S %z'))

In [ ]:
print(gb_clf.best_param)
print(gb_clf.best_model_kpi)
print(gb_clf.best_driver.head(51))
gb_clf._modeltest(input_train)

#shutil.copy('lift_chart.png',"lift_chart_"+oversample+str(len(input_train)) + gb_clf.mode + ".png")
#shutil.copy('lorenz_curve.png',"lorenz_curve_"+oversample+str(len(input_train)) + gb_clf.mode + ".png")
#wb = xlsxwriter.Workbook("Result_"+ fn_idc+"_"+str(len(input_train))+"_" + gb_clf.mode+"_graph.xlsx")
wb = xlsxwriter.Workbook("Train_Graph_"+str(len(input_train))+"_" + datetime.now().strftime("%H%M%S") +".xlsx")
#wb = xlsxwriter.Workbook("test.xlsx")
sht = wb.add_worksheet('Chart')
sht.insert_image('D4','lift_chart.png')
sht.insert_image('D24','lorenz_curve.png')
#sht2 = wb.add_worksheet('lorenz_curve')
wb.close()

In [ ]:
fw = pd.ExcelWriter("Train_Parm_"+ str(len(input_train))+"_" + datetime.now().strftime("%H%M%S") +".xlsx")
#fw = pd.ExcelWriter("Result_"+ fn_idc+"_"+str(len(input_train))+"_" + gb_clf.mode+"_param.xlsx")
#fw = pd.ExcelWriter("test.xlsx")
gb_clf.best_param.to_excel(fw,"best_parm")
gb_clf.best_model_kpi.to_excel(fw,'kpi')
gb_clf.best_driver.to_excel(fw,'best_driver')
gb_clf.lift_table.to_excel(fw,'lift_table')

In [ ]:
gb_clf._modeltest(input_valid)

In [ ]:
gb_clf.best_model

In [ ]:
csv_predict='/Users/wyatt/Documents/Docs-wMBP15/SAS/stk_model/predict_base_20170727.csv'
input_predict = pd.DataFrame(pd.read_csv(csv_predict)).fillna(0)
input_predict['month']=20170727
input_predict.head()

In [ ]:
input_predict.info()
x_valid_predict=input_predict.drop(['cust_id'], axis=1).values

In [ ]:
p_predict_scored = pd.DataFrame(gb_clf.best_model.predict_proba(x_valid_predict))

In [ ]:
tmp=pd.concat([input_predict,p_predict_scored],axis=1)

In [ ]:
tmp.rename(columns={1:'score'},inplace=True)

In [ ]:
tmp=tmp.sort_values(by='score',ascending=False)

In [ ]:
tmp.head()

In [ ]:
tmp[['month','cust_id','score']].head(10)

In [ ]:
### data preparation
print('Data load strs at '+ datetime.now(pytz.timezone('Asia/Hong_Kong')).strftime('%y-%m-%d %H:%M:%S %z'))

csv_valid_ot1='/Users/wyatt/Documents/Docs-wMBP15/SAS/stk_model/model.Wyatt_stock_full_train.csv'
csv_valid_ot2='/Users/wyatt/Documents/Docs-wMBP15/SAS/stk_model/model.Wyatt_stock_full_valid.csv'

input_valid_ot1=pd.DataFrame(pd.read_csv(csv_valid_ot1))
input_valid_ot2=pd.DataFrame(pd.read_csv(csv_valid_ot2))
input_valid_ot1=input_valid_ot1.fillna(0)
input_valid_ot2=input_valid_ot2.fillna(0)
print('Data load ends at '+ datetime.now(pytz.timezone('Asia/Hong_Kong')).strftime('%y-%m-%d %H:%M:%S %z'))
print('Data as of '+ datetime.now(pytz.timezone('Asia/Hong_Kong')).strftime('%y-%m-%d %H:%M:%S %z'))
print("#------------------- Separate Line 3 -------------------# ")
print("Valid Out-time Data:")
input_valid_ot1.info()
print("#------------------- Separate Line 4 -------------------# ")
print("Valid Out-time Data:")
input_valid_ot2.info()
print("#------------------- Separate Line 4 -------------------# ")

In [ ]:
gb_clf._modeltest(input_valid_ot1)

In [ ]:
gb_clf._modeltest(input_valid_ot2)